In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tweet = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')
tweet.head()

In [ ]:
tweet.shape

In [ ]:
tweet.describe().transpose()

In [ ]:
tweet.info()

In [ ]:
tweet.columns

In [ ]:
tweet.isnull().sum()

In [ ]:
tweet.nunique()

In [ ]:
tweet = tweet.drop(['retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location','name','user_timezone'], axis = 1)
tweet.head()

In [ ]:
tweet['negativereason'] = tweet['negativereason'].fillna('')
tweet['negativereason_confidence'] = tweet['negativereason_confidence'].fillna(0)
tweet['airline_sentiment_gold'] = tweet['airline_sentiment_gold'].fillna('')
tweet['negativereason_gold'] = tweet['negativereason_gold'].fillna('')
tweet.isnull().sum()

In [ ]:
print('The negative reasons are as follows: ')
tweet['negativereason'].unique()

In [ ]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import string
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = tweet[['text','airline_sentiment']]
data.head()

In [ ]:
stop_words = stopwords.words('english')
punc = string.punctuation
stemmer = PorterStemmer()

In [ ]:
X=data['text']
y=data['airline_sentiment']
import re
cleaned_data=[]
for i in range(len(X)):
  tweets = re.sub('[^a-zA-Z]',' ',X.iloc[i])
  tweets = tweets.lower().split()
  tweets = [stemmer.stem(word) for word in tweets if (word not in stop_words) and (word not in punc)]
  tweets = ' '.join(tweets)
  cleaned_data.append(tweets)

In [ ]:
cleaned_data

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15,5))
plt.hist(y)
plt.xlabel('Sentiment')
plt.ylabel('Count')

In [ ]:
sentiment_ordering = ['positive', 'neutral', 'negative']

y = y.apply(lambda x: sentiment_ordering.index(x))

In [ ]:
y

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000,stop_words=['virginamerica','unit'])
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model=MultinomialNB()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_fin,y,test_size=0.3)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
cf = classification_report(y_test,y_pred)
print(cf)